In [ ]:
from datasets import load_dataset

dataset_path = "Qwen2-Magpie-Pro-1M-v0.1.jsonl" # Put your path to the dataset here
dataset = load_dataset("json", data_files=dataset_path)
print(dataset)

In [ ]:
def high_quality_filter(example):
    return (
        example['input_quality'] in ['good', 'excellent']
        and example['instruct_reward'] > -10
        and not example['instruction'].endswith(':')
        and (
            example['min_similar_conversation_id'] is None
            or example['conversation_id'] == example['min_similar_conversation_id']
        )
    )
filtered_dataset = dataset['train'].filter(high_quality_filter)
print(filtered_dataset)

In [ ]:
def get_output_length(example):
    return len(example['response'])

filtered_dataset = filtered_dataset.map(lambda x: {'response_length': get_output_length(x)})
# sort the dataset by response_length
sorted_dataset = filtered_dataset.sort('response_length', reverse=True)
# select the top 300000 examples
top_x_dataset = sorted_dataset.select(range(300000))
# Shuffle the dataset
top_x_dataset = top_x_dataset.shuffle(seed=42)

In [ ]:
# Submit the dataset to the hub
top_x_dataset.push_to_hub("Magpie-Align/Magpie-Qwen-Pro-300K-Filtered", private=True)